# [Mohammad Shaheer Siddiqi]
# Programming Exercise \#3
---

# Preamble

In [8]:
# optional code cell when using Google Colab with Google Drive

# remove the docstring comment block below in order to mount Google Drive
'''
# mount Google Drive in Google Colab
from google.colab import drive
drive.mount('/content/drive')

# change directory using the magic command %cd
### replace [MY PATH] below with your own path in Google Drive ###
### %cd /content/drive/My\ Drive/[MY PATH] ###
'''

"\n# mount Google Drive in Google Colab\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\n# change directory using the magic command %cd\n### replace [MY PATH] below with your own path in Google Drive ###\n### %cd /content/drive/My\\ Drive/[MY PATH] ###\n"

In [9]:
# import relevant Python libraries

### Your import commands go here ###

# **1. Noise Level Classification Using Plug-in Classifiers**

## 1.1

### (a)

In [1]:
import numpy as np
import pandas as pd

train = pd.read_csv("NoiseClassificationTrainingData.csv")

#Separate by class
x0 = train[train["ClassLabel"] == 0]["NoiseLevel"].values
x1 = train[train["ClassLabel"] == 1]["NoiseLevel"].values
x2 = train[train["ClassLabel"] == 2]["NoiseLevel"].values

#means
mu0 = np.mean(x0)
mu1 = np.mean(x1)
mu2 = np.mean(x2)

#Common variance 
sigma2 = (
    np.sum((x0 - mu0)**2) +
    np.sum((x1 - mu1)**2) +
    np.sum((x2 - mu2)**2)
) / len(train)

print(mu0, mu1, mu2, sigma2)

44.65630954576823 55.277231807111434 64.88690064177815 11.995287584605006


### (b)

[The Gaussian model with class specific means and a shared variance appears reasonably appropriate for the NoiseLevel feature. For each class, the histogram is roughly unimodal and approximately symmetric around its estimated mean, and the overlaid Gaussian PDFs track the main bulk of the data well. While there are small deviations in the tails and minor asymmetries that reflect sampling variability and real-world noise, the spreads of the three classes are similar enough that the common variance assumption is plausible.]

## 1.2

### (a)

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm

plt.figure(figsize=(10,6))

bins = 20

#Plot histograms
plt.hist(x0, bins=bins, density=True, alpha=0.4, label="Class 0 Hist")
plt.hist(x1, bins=bins, density=True, alpha=0.4, label="Class 1 Hist")
plt.hist(x2, bins=bins, density=True, alpha=0.4, label="Class 2 Hist")

#PDF plotting
xmin = min(train["NoiseLevel"])
xmax = max(train["NoiseLevel"])
xx = np.linspace(xmin, xmax, 500)

plt.plot(xx, norm.pdf(xx, mu0, np.sqrt(sigma2)), label="Class 0 PDF")
plt.plot(xx, norm.pdf(xx, mu1, np.sqrt(sigma2)), label="Class 1 PDF")
plt.plot(xx, norm.pdf(xx, mu2, np.sqrt(sigma2)), label="Class 2 PDF")

plt.xlabel("Noise Level (dB)")
plt.ylabel("Density")
plt.legend()
plt.title("Histograms + Gaussian PDFs")
plt.show()

### (b)

_[The histograms for each class appear roughly bell shaped and centered around their means. The shared variance Gaussian PDFs overlay the histograms reasonably well. Although there are small deviations due to sample size, the assumption of Gaussian distributions with different means but common variance is seen.]_

## 1.3

In [3]:
priors = train["ClassLabel"].value_counts(normalize=True).sort_index()
print(priors)

ClassLabel
0.0    0.2625
1.0    0.2200
2.0    0.5175
Name: proportion, dtype: float64


## 1.4

In [4]:
import numpy as np
import pandas as pd
from scipy.stats import norm

test = pd.read_csv("NoiseClassificationTestData.csv")
x_test = test["NoiseLevel"].values          
y_test = test["ClassLabel"].values          

mus = np.array([mu0, mu1, mu2])             
sigma = np.sqrt(sigma2)                     

pi = priors.values                         

#Loss 
L = np.array([
    [-1, 2, 4],
    [ 2, 0, 4],
    [ 4, 4, 0]
])   

def summarize_classifier(name, y_true, y_pred):
    print(f"\n===== {name} =====")
    
    pred_counts = pd.Series(y_pred).value_counts().sort_index()
    print("Predicted counts per class:")
    for c in range(3):
        print(f"  Class {c}: {pred_counts.get(c, 0)}")
    
    print("\nCorrectly classified per true class:")
    for c in range(3):
        mask = (y_true == c)
        correct = np.sum(y_pred[mask] == c)
        total = np.sum(mask)
        print(f"  True class {c}: {correct} / {total}")
        
#N_samples, 3
likelihoods = norm.pdf(x_test[:, None], loc=mus[None, :], scale=sigma)

#ML classifier
y_pred_ML = np.argmax(likelihoods, axis=1)
summarize_classifier("ML classifier", y_test, y_pred_ML)

#MAP
posterior_unnorm = likelihoods * pi[None, :]        
y_pred_MAP = np.argmax(posterior_unnorm, axis=1)
summarize_classifier("MAP classifier", y_test, y_pred_MAP)

posterior = posterior_unnorm / posterior_unnorm.sum(axis=1, keepdims=True)

risks = posterior @ L.T

y_pred_Bayes = np.argmin(risks, axis=1)
summarize_classifier("Plug-in Bayes classifier", y_test, y_pred_Bayes)


===== ML classifier =====
Predicted counts per class:
  Class 0: 22
  Class 1: 28
  Class 2: 50

Correctly classified per true class:
  True class 0: 21 / 27
  True class 1: 21 / 22
  True class 2: 50 / 51

===== MAP classifier =====
Predicted counts per class:
  Class 0: 23
  Class 1: 25
  Class 2: 52

Correctly classified per true class:
  True class 0: 22 / 27
  True class 1: 20 / 22
  True class 2: 51 / 51

===== Plug-in Bayes classifier =====
Predicted counts per class:
  Class 0: 26
  Class 1: 22
  Class 2: 52

Correctly classified per true class:
  True class 0: 25 / 27
  True class 1: 20 / 22
  True class 2: 51 / 51


## 1.5

### (a)

In [5]:
def misclassification_error(y_true, y_pred):
    return np.mean(y_true != y_pred)

err_ML   = misclassification_error(y_test, y_pred_ML)
err_MAP  = misclassification_error(y_test, y_pred_MAP)
err_Bayes = misclassification_error(y_test, y_pred_Bayes)

print("Misclassification Errors (Zero-One Loss):")
print(f"  ML Error:    {err_ML:.4f}")
print(f"  MAP Error:   {err_MAP:.4f}")
print(f"  Bayes Error: {err_Bayes:.4f}")

Misclassification Errors (Zero-One Loss):
  ML Error:    0.0800
  MAP Error:   0.0700
  Bayes Error: 0.0400


### (b)

In [7]:
def general_loss_risk(y_true, y_pred, L):
    y_true = np.asarray(y_true, dtype=int)
    y_pred = np.asarray(y_pred, dtype=int)
    losses = L[y_pred, y_true]        # Now indexing is valid
    return np.mean(losses)

risk_ML    = general_loss_risk(y_test, y_pred_ML, L)
risk_MAP   = general_loss_risk(y_test, y_pred_MAP, L)
risk_Bayes = general_loss_risk(y_test, y_pred_Bayes, L)

print("General Loss Risks:")
print(f"  ML Risk:    {risk_ML:.4f}")
print(f"  MAP Risk:   {risk_MAP:.4f}")
print(f"  Bayes Risk: {risk_Bayes:.4f}")


General Loss Risks:
  ML Risk:    -0.0300
  MAP Risk:   -0.0600
  Bayes Risk: -0.1500


## 1.6

### (a)

_[The MAP classifier is designed to minimize the expected zero–one loss, since choosing the class with the largest posterior probability minimizes the probability of error under 0–1 loss.]_

### (b)

_[The plug in Bayes classifier with the given loss matrix is designed to minimize the expected general loss. For each input x, it computes the expected loss of predicting class j as R(j|x) = 2SUMk=0 L(j,k) P(k|x), and selects smallest value of R(j|x).]_

### (c)

_[The empirical results only partially match the theoretical guarantees. The MAP classifier should have the lowest misclassification error, but in our results the Bayes classifier achieves the smallest error .04, followed by MAP .07 and ML .08. This means MAP does not empirically minimize the zero one loss in this dataset. For the general loss, the plug in Bayes classifier is the optimal case, but the ML classifier actually obtains the lowest empirical loss −0.280. Only the relative ordering of MAP > ML for zero one loss aligns with theory, while the expected loss results do not entirely match those theoretical expectations.]_

### (d)

_[The plug in Bayes classifier with the given loss matrix is designed to minimize the expected general loss. It does this by choosing the class that gives the smallest expected loss]_